# Modeling a Double Pendulum
By Marion Madanguit and Nicola van Moon

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

import math

## Question
The double pendulum is known to be one of the most chaotic mechanical systems in existence. Its motion is characterized by rich dynamic swinging in all degrees and directions. Although seemingly erratic, the motion of the double pendulum can be predicted with an understanding of the physical laws governing it.

Our primary goal is to create a model of the double pendulum. With our model, we hope to be able to predict the motion of the double pendulum given any initial position. In other words, we are aiming to answer the following question: Given the initial positions of the two masses on a double pendulum, what motion does the system undergo? Having a model of this system will not only offer us an interesting visual experience but it will allow us to graphically plot the impact of slight changes in initial position and perhaps give us a greater understanding of the chaos of the double pendulum.

## Model
To answer this question, we first defined our system. Our system would consist of two masses connected by rods in a double pendulum setup as shown in the diagram below. We assumed the rods were massless and the masses were point masses for simplification purposes. This assumption is relatively accurate if the rods are made of a lightweight material and the masses are made of a dense material.

We then created force diagrams for the two masses. Each mass is acted on by a force of gravity and the forces of tension from the rod(s) touching it. We assumed that friction and air resistance were negligible to the system. A well-made double pendulum usually has low friction about its pivots so ignoring friction is a somewhat reasonable assumption for this system. Air resistance also has a small effect at such low velocities. It is important to note, however, that in a real-world scenario, air resistance and friction would ultimately cause the double pendulum to slow to a stop.

![Schematic](Images/Diagrams.png)
![Question](Text1.PNG)
![Question](2Text.PNG)
![Question](3Text.PNG)

In [2]:
m = 1
s = 1
minute = 1
day = 1
kg = 1

1

In [3]:
t_0 = 0 * s
t_end = 5 * s
dt = 0.01 * s

params = Params(L1 = 1 * m,
                L2 = 1 * m,
                M1 = 1 * kg,
                M2 = 1 * kg,
                theta1_init = 80 * pi / 180, # in rad
                theta2_init = -80 * pi / 180, # in rad
                omega1_init = 0 / s, # in rad / s
                omega2_init = 0 / s, # in rad / s
                g = 9.8 * m / s**2,
                t_0=t_0, t_end=t_end, dt=dt)

,values
L1,1.000000
L2,1.000000
M1,1.000000
M2,1.000000
theta1_init,1.396263
theta2_init,-1.396263
omega1_init,0.000000
omega2_init,0.000000
g,9.800000
t_0,0.000000


In [4]:
def make_system(params):

    # angular positions
    theta1 = params.theta1_init
    theta2 = params.theta2_init

    # angular velocities
    omega1 = params.omega1_init
    omega2 = params.omega2_init

    init = State(theta1 = theta1, theta2 = theta2, omega1 = omega1, omega2 = omega2)

    return System(params, init=init)

system = make_system(params)

,values
L1,1
L2,1
M1,1
M2,1
theta1_init,1.39626
theta2_init,-1.39626
omega1_init,0
omega2_init,0
g,9.8
t_0,0


In [5]:
def slope_func(state, t, system):

    theta1, theta2, omega1, omega2 = state
    M1, M2, g = system.M1, system.M2, system.g
    L1, L2 = system.L1, system.L2

    dtheta1dt = omega1
    dtheta2dt = omega2

    #equations for the angular acceleration of each mass
    domega1dt = (-g * (2*M1 + M2) * math.sin(theta1) - M2 * g * math.sin(theta1 - 2*theta2) - 2 * math.sin(theta1 - theta2) * M2 *(L2*omega2**2 + L1*math.cos(theta1 - theta2)*omega1**2)) / (L1 * (2*M1 + M2 - M2*math.cos(2*theta1-2*theta2)))
    domega2dt = (2 * math.sin(theta1-theta2) * ((M1+M2)*L1*omega1**2 + g*(M1+M2)*math.cos(theta1) + L2*M2*math.cos(theta1-theta2)*omega2**2)) / (L2 * (2*M1 + M2 - M2*math.cos(2*theta1-2*theta2)))

    return dtheta1dt, dtheta2dt, domega1dt, domega2dt

In [6]:
def event_func(state, t, system):

    return system.t_end - t #equals 0 at time t so the simulation ends

In [7]:
results, details = run_ode_solver(system, slope_func, events=event_func)
details

,values
success,True
message,The solver successfully reached the end of the...


In [8]:
results

,theta1,theta2,omega1,omega2
0.00,1.396263,-1.396263,0.000000,0.000000
0.01,1.395805,-1.396211,-0.091607,0.010439
0.02,1.394431,-1.396054,-0.183116,0.021057
0.03,1.392143,-1.395790,-0.274428,0.032036
0.04,1.388943,-1.395413,-0.365444,0.043560
...,...,...,...,...
4.96,-1.321228,0.916788,-0.682353,2.847858
4.97,-1.327423,0.945249,-0.556586,2.845809
4.98,-1.332359,0.973704,-0.430670,2.846633
4.99,-1.336036,1.002181,-0.304672,2.849851


In [9]:
def pol_vector(mag, angle, offset, direction): #offset and direction allow for angles not measured counter-clockwise from the positive x-axis to be acurately turned into the correct vectors
    x, y = pol2cart(direction*angle + offset, mag)
    return Vector(x, y)

In [10]:
def get_R1(theta): #R1 is the position of the inner mass in reference to the origin
    return pol_vector(system.L1, theta, 3*pi/2, 1) #3pi/2 because it is measured from the negative y axis, 1 becasue it is counter-clockwise

def get_R2(theta): #R2 is the position of the outer mass in reference to the inner mass
    return pol_vector(system.L2, theta, 3*pi/2, -1) #3pi/2 because it is measured from the negative axis, -1 because it is measured clockwise

def get_R3(R1, R2): #R3 is the position of the outer mass in reference to the origin
    return R1 + R2

In [11]:
def vectorize_results(results):
    get_R1_vectorized = np.vectorize(get_R1) # numpy vectorize transforms function so that it can operate on and return numpy arrays
    get_R2_vectorized = np.vectorize(get_R2)
    get_R3_vectorized = np.vectorize(get_R3)

    results['R1'] = get_R1_vectorized(results['theta1'])
    results['R2'] = get_R2_vectorized(results['theta2'])
    results['R3'] = get_R3_vectorized(results['R1'], results['R2'])
    return results

results = vectorize_results(results)

,theta1,theta2,omega1,omega2,R1,R2,R3
0.00,1.396263,-1.396263,0.000000,0.000000,"[0.984807753012208 dimensionless, -0.173648177...","[0.984807753012208 dimensionless, -0.173648177...","[1.969615506024416 dimensionless, -0.347296355..."
0.01,1.395805,-1.396211,-0.091607,0.010439,"[0.984728106134212 dimensionless, -0.174099273...","[0.9847987032357167 dimensionless, -0.17369949...","[1.9695268093699287 dimensionless, -0.34779876..."
0.02,1.394431,-1.396054,-0.183116,0.021057,"[0.9844879826217118 dimensionless, -0.17545202...","[0.9847714302144067 dimensionless, -0.17385404...","[1.9692594128361185 dimensionless, -0.34930607..."
0.03,1.392143,-1.395790,-0.274428,0.032036,"[0.9840839276351747 dimensionless, -0.17770431...","[0.9847254154737645 dimensionless, -0.17411449...","[1.9688093431089393 dimensionless, -0.35181880..."
0.04,1.388943,-1.395413,-0.365444,0.043560,"[0.9835102102615904 dimensionless, -0.18085260...","[0.9846597863286497 dimensionless, -0.17448525...","[1.9681699965902402 dimensionless, -0.35533786..."
...,...,...,...,...,...,...,...
4.96,-1.321228,0.916788,-0.682353,2.847858,"[-0.9690190092831374 dimensionless, -0.2469861...","[-0.7936517813092249 dimensionless, -0.6083722...","[-1.7626707905923622 dimensionless, -0.8553584..."
4.97,-1.327423,0.945249,-0.556586,2.845809,"[-0.9705305082918401 dimensionless, -0.2409782...","[-0.8106424887852356 dimensionless, -0.5855414...","[-1.7811729970770758 dimensionless, -0.8265197..."
4.98,-1.332359,0.973704,-0.430670,2.846633,"[-0.9717082865176763 dimensionless, -0.2361842...","[-0.8269738559596683 dimensionless, -0.5622403...","[-1.7986821424773445 dimensionless, -0.7984246..."
4.99,-1.336036,1.002181,-0.304672,2.849851,"[-0.9725701234883731 dimensionless, -0.2326098...","[-0.8426471841869626 dimensionless, -0.5384660...","[-1.8152173076753357 dimensionless, -0.7710759..."


In [12]:
def plot_results(results, Title):
    R1_x = results.R1.extract('x')
    R1_y = results.R1.extract('y')
    R3_x = results.R3.extract('x')
    R3_y = results.R3.extract('y')

    plot(R1_x, R1_y, color='C2')
    plot(R3_x, R3_y, color='C3')

    decorate(title= Title,
             xlabel='x position (m)',
             ylabel='y position (m)')

plot_results(results, 'Double Pendulum (80 degrees)')

## Results
To answer our initial question, we ran our stimulation for 5 different starting angles (161 to 165 degrees) and plotted the motion of the two masses over a period of 5 seconds. We chose to make only slight changes to its initial position to see if and how those small differences would impact the motion of the double pendulum. The motion of the first mass is represented by a green line and the motion of the second mass is represented by a red line.

In [13]:
def run_with_angle(angle):
    params = Params(L1 = 1 * m,
                    L2 = 1 * m,
                    M1 = 1 * kg,
                    M2 = 1 * kg,
                    V1_init = 0 * m / s,
                    V3_init = 0 * m / s,
                    theta1_init = angle * pi / 180, # in rad
                    theta2_init = -1*angle * pi / 180, # in rad
                    omega1_init = 0 / s, # in rad / s
                    omega2_init = 0 / s, # in rad / s
                    g = 9.8 * m / s**2,
                    t_0=t_0, t_end=t_end, dt=dt)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    results = vectorize_results(results)
    plot_results(results, f"{angle} degrees")

In [14]:
run_with_angle(161)

In [15]:
run_with_angle(162)

In [16]:
run_with_angle(163)

In [17]:
run_with_angle(164)

In [18]:
run_with_angle(165)

## Interpretation 
Our simulation confirms the chaotic motion of a double pendulum. With even a 1-degree change in initial angle, the motion of the outer mass is completely different.

By not including friction or air resistance in our model, we simulated an undampened system which means the pendulum will never come to a stop. The lack of dampening forces also means that our model is somewhat inaccurate. The longer we run the simulation, the more inaccurate it gets because the motion continues with the same energy rather than becoming increasingly dampened. Additionally, the small changes that friction would have made to the pendulum's motion earlier in the simulation may lead to unpredictably inaccurate results as time goes on due to the chaotic motion of the pendulum. We could add a friction force to model a more realistic dampened, however, our friction force would have to be completely accurate in order to avoid the compounding inaccuracy that comes from the chaos of the system.

Our model is successful at showing the chaos of the system and what the motion of a double pendulum could look like, but it would be bad at predicting the motion of a real-life double pendulum. Any tiny differences between our model and the real system based on our assumptions or imprecision in the physical system would create large discrepancies between our predicted behavior and the real motion as the simulation got further in time.

It was extremely difficult to iterate within our model. To model the system, even in its simplest form, required incredibly complex equations. The main iteration our project underwent occurred during the equation-writing portion of the project. Before attempting to understand the motion equations for the double pendulum, we looked at equations for a single pendulum. The motions equations for the single pendulum were grounded by the same physical laws as the double pendulum but were far less complicated. Our understanding of the single pendulum equations allowed us to better understand the double pendulum equations and validate them in many cases. In the future, we could iterate upon our model to eliminate some of our assumptions and make it more accurate.

## External Validity Experiment
Due to the chaos of the system and the assumptions discussed in our interpretation section, our model is unlikely to match the behavior of a double pendulum in real life. However, our simulation could be validated if we ran a real-world double pendulum under certain conditions, like a short time period or a small initial angle. Either of those two conditions would produce less chaotic motion that would be less impacted by the inaccuracy our assumptions create.

For a simple validation experiment, we could build a double pendulum with light rods and dense masses and release it from rest at a 100-degree angle. We could film the motion and trace the movement of the two masses over time. To validate our model, we could compare this path to our simulated path. If they were similar for the first few seconds (before the inaccuracies from friction compound and become substantial) we would know that our model works.

## References
http://www.phys.lsu.edu/faculty/gonzalez/Teaching/Phys7221/DoublePendulum.pdf

https://www.myphysicslab.com/pendulum/double-pendulum-en.html

https://www.myphysicslab.com/pendulum/double-pendulum-en.html